# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and sub-folder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))

/Users/theomar/data-engineering-udacity/Project2


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csv_reader = csv.reader(csvfile) 
        next(csv_reader)
        
 # extracting each data row one by one and append it        
        for line in csv_reader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if row[0] == '':
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

### Lets create a keyspace with simple strategy and replication factor 3. So the data will be replicated into three nodes in total for high availability. 

In [6]:
session.execute("CREATE KEYSPACE if not exists event WITH REPLICATION = { 'class' : 'SimpleStrategy', 'replication_factor' : 3 }")

#### Set Keyspace

In [7]:
session.set_keyspace('event')

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




### Now create a table called query_by_sessionId_and_itemInSession for the fields sessionId, itemInSession,artist, song_title, song_length. We can use sessionId and itemInSession as a composite key since our query parameter are those two. Both are unique value, so we can go with that. 

In [8]:
session.execute("create table if not exists query_by_sessionId_and_itemInSession(sessionId int,itemInSession int,artist text, song_title text,song_length double,primary key((sessionId,itemInSession)))")                    

### Lets read the csv file, find the data.  And finally into into our cassandra table `query_by_sessionId_and_itemInSession`

In [9]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'
with open(file, encoding = 'utf8') as f:
    csv_reader = csv.reader(f)
    next(csv_reader) # skip header
    for line in csv_reader:
        artist=line[0]
        sessionId=int(line[8])
        itemInSession=int(line[3])
        song_title=line[9]
        song_length=float(line[5])        
        query = """INSERT INTO query_by_sessionId_and_itemInSession (sessionId,itemInSession,artist, song_title,song_length)VALUES (%s, %s, %s, %s, %s)"""
        session.execute(query,(sessionId, itemInSession, artist,song_title,song_length))        

#### Do a SELECT to verify that the data have been inserted into each table

In [10]:
rows= session.execute("SELECT artist, song_title FROM query_by_sessionId_and_itemInSession LIMIT 10")
for row in rows:
    print(row)

Row(artist='Los Rodriguez', song_title='Enganchate Conmigo')
Row(artist='The Gerbils', song_title='(iii)')
Row(artist='De-Phazz', song_title='Astrud Astronette')
Row(artist='Tom Waits', song_title="Whistlin' Past The Graveyard")
Row(artist='Jimmy Eat World', song_title='The Middle')
Row(artist='Jack Johnson', song_title='Better Together')
Row(artist='Moose', song_title='Do You Remember')
Row(artist='Fleetwood Mac', song_title='World Turning (LP Version)')
Row(artist='Goldfrapp', song_title='Lovely Head')
Row(artist='Pearl Jam', song_title='Encore Break')


### Run the first query to achieve the result for the requirements `Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4`

In [11]:
rows= session.execute("SELECT artist, song_title FROM query_by_sessionId_and_itemInSession where sessionId=%s and itemInSession=%s",[338,4])
print(rows[0])

Row(artist='Faithless', song_title='Music Matters (Mark Knight Dub)')


/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: ResultSet indexing support will be removed in 4.0. Consider using ResultSet.one() to get a single row.
  


## Second Query

### Create table called `query_by_userid_and_sessionid` with the fields named sessionId, userId, itemInSession, artist, song_title, first_name, last_name. Based on our query, we can use userId, sessionId as composite key and itemInSession as clustering key for the sorting. 



In [12]:
session.execute("create table if not exists query_by_userid_and_sessionid(sessionId int,userId int,itemInSession int,artist text, song_title text,first_name text,last_name text,primary key((userId,sessionId),itemInSession))")

### Read the csv file, extract the field from each line and insert into the table `query_by_userid_and_sessionid
`

In [13]:
file = 'event_datafile_new.csv'
with open(file, encoding = 'utf8') as f:
    csv_reader = csv.reader(f)
    next(csv_reader) # skip header
    for line in csv_reader:
        userId=int(line[10])
        first_name=line[1]
        last_name=line[4]
        artist=line[0];
        sessionId=int(line[8])
        itemInSession=int(line[3])
        song_title=line[9]      
        query = """INSERT INTO query_by_userid_and_sessionid (sessionId,userId,itemInSession,artist, song_title,first_name,last_name)VALUES (%s, %s, %s, %s,%s,%s,%s)"""
        session.execute(query,(sessionId, userId,itemInSession, artist,song_title,first_name,last_name))
                    

### Verify that data has been inserted properly

In [14]:
rows= session.execute("SELECT song_title,first_name, last_name FROM query_by_userid_and_sessionid LIMIT 10")
for row in rows:
    print(row)

Row(song_title='Sad Statue', first_name='Emily', last_name='Benson')
Row(song_title='Stranger Lover', first_name='Emily', last_name='Benson')
Row(song_title='Zero', first_name='Emily', last_name='Benson')
Row(song_title='Head Up (LP Version)', first_name='Kinsley', last_name='Young')
Row(song_title='Playa Hater (Amended Version)', first_name='Kinsley', last_name='Young')
Row(song_title='Stay (The Black Rose And The Universal Wheel)', first_name='Kinsley', last_name='Young')
Row(song_title="Roxane's Veil", first_name='Kinsley', last_name='Young')
Row(song_title='TRUE', first_name='Kinsley', last_name='Young')
Row(song_title="I'm Done", first_name='Kinsley', last_name='Young')
Row(song_title="I'm Down", first_name='Kinsley', last_name='Young')


### Run the second query from the requirements above. 

In [15]:
rows= session.execute("SELECT song_title,first_name, last_name FROM query_by_userid_and_sessionid where userId=%s and sessionId=%s",[10,182])
for row in rows:
    print(row)

Row(song_title="Keep On Keepin' On", first_name='Sylvie', last_name='Cruz')
Row(song_title='Greece 2000', first_name='Sylvie', last_name='Cruz')
Row(song_title='Kilometer', first_name='Sylvie', last_name='Cruz')
Row(song_title='Catch You Baby (Steve Pitron & Max Sanna Radio Edit)', first_name='Sylvie', last_name='Cruz')


# Third Query

### Create table with the column named first_name, last_name,song_title

In [16]:
session.execute("create table if not exists query_by_song(first_name text,last_name text,song_title text,primary key((song_title),first_name,last_name))")




### Read the csv file, extract the data from their. Finally insert into the table called `query_by_song`

In [17]:
file = 'event_datafile_new.csv'
with open(file, encoding = 'utf8') as f:
    csv_reader = csv.reader(f)
    next(csv_reader) # skip header
    for line in csv_reader:
        first_name=line[1]
        last_name=line[4]
        song_title=line[9]      
        query = """INSERT INTO query_by_song (first_name,last_name,song_title)VALUES (%s, %s, %s)"""
        session.execute(query,(first_name,last_name,song_title))
                

### Check the data has been inserted into the table properly

In [18]:
rows= session.execute("SELECT first_name, song_title, last_name FROM query_by_song LIMIT 10")
for row in rows:
    print(row)

Row(first_name='Tegan', song_title='Victoria (LP Version)', last_name='Levine')
Row(first_name='Chloe', song_title='Soy AlbaÃ\x83Â±il', last_name='Cuevas')
Row(first_name='Aleena', song_title='Snow [Hey Oh] (Album Version)', last_name='Kirby')
Row(first_name='Matthew', song_title='Supermassive Black Hole (Twilight Soundtrack Version)', last_name='Jones')
Row(first_name='Aleena', song_title='Hold On', last_name='Kirby')
Row(first_name='Aleena', song_title="Words- Till the mornin' comes Mix", last_name='Kirby')
Row(first_name='Tegan', song_title='Breathless', last_name='Levine')
Row(first_name='Jacob', song_title="But Now I'm Black", last_name='Klein')
Row(first_name='Amiya', song_title='Canada', last_name='Davidson')
Row(first_name='Chloe', song_title='Quien Te Dijo', last_name='Cuevas')


### Run the third query based on the requirements above

In [19]:
rows= session.execute("SELECT first_name, song_title, last_name FROM query_by_song where song_title=%s",['All Hands Against His Own'])
for row in rows:
    print(row)

InvalidRequest: Error from server: code=2200 [Invalid query] message="Cannot execute this query as it might involve data filtering and thus may have unpredictable performance. If you want to execute this query despite the performance unpredictability, use ALLOW FILTERING"

### Drop the tables before closing out the sessions

In [22]:
## TO-DO: Drop the table before closing out the sessions

In [23]:
session.execute("DROP TABLE query_by_sessionId_and_itemInSession;")
session.execute("DROP TABLE query_by_userid_and_sessionid;")
session.execute("DROP TABLE query_by_song;")

### Close the session and cluster connection¶

In [24]:
session.shutdown()
cluster.shutdown()